In [1]:
import sys
sys.path.append('../../../../prediction')

import numpy as np
import pandas as pd

from putils.observation import compute_rmse, compute_mae, compute_mape, load_data_from_tuned_folder

In [2]:
# Get data from df
df = load_data_from_tuned_folder('l2_updated')
df = df.drop(columns=["Time", "Actual"])
alpha = 100

# Dict of models with their initial weights (1 / number of models)
weights = {}
for column in df.columns:
    if column != "L3_WEIGHTED_AVERAGE" and column != "Raw":
        weights[column] = 1 / (len(df.columns) - 1) # -1 because the column "Raw" is ignored

# Create a new column for the average value
df["L3_WEIGHTED_AVERAGE"] = None

# Loop through each row and compute the weighted average
count = 0
for index, row in df.iterrows():
    # Find the average value with weight of the row 
    total = 0
    for column in df.columns:
        if column != "L3_WEIGHTED_AVERAGE" and column != "Raw":
            total += row[column] * weights[column]

    # Add the average value to the row
    df.loc[index, "L3_WEIGHTED_AVERAGE"] = total
    # Update the weights every 5 rows
    if count % 5 == 0:
        # Compute RMSE of each model from 0 to count - 1
        rmses = {}
        for column in df.columns:
            if column != "L3_WEIGHTED_AVERAGE" and column != "Raw":
                rmses[column] = compute_rmse(df["Raw"][:count], df[column][:count])
        # Compute the sum of exponential RMSEs [exp(alpha * -rmse)]
        rmse_sum = sum([np.exp(alpha * -rmse) for rmse in rmses.values()])
        # Update the weights
        for column in df.columns:
            if column != "L3_WEIGHTED_AVERAGE" and column != "Raw":
                result = np.exp(alpha * -rmses[column]) / rmse_sum
                # Check if nan
                if np.isnan(result) == False:
                    weights[column] = result
    # Increment count
    count += 1

print(f"final weights: {weights}")

# Save the data into new file
df.to_csv("l3_tunned_random_forest.csv")

final weights: {'LINEAR_REGRESSION': 0.3428161609296413, 'RANDOM_FOREST': 0.3321966271666997, 'FEEDFORWARD_NEURAL_NETWORK': 0.32498721190365903}
